In [1]:
%load_ext autoreload
%autoreload 2

### Load constants

In [2]:
from src.constants import (
    RAW_DIR_PATH,
    PROCESSED_DIR_PATH
)

### Load libs

In [3]:
import os
import pymongo
import pandas as pd
from datetime import datetime as dt
from src.features.dedup_model import load_results

In [4]:
DB_USERNAME = os.getenv("MONGO_USERNAME")
DB_PASSWORD = os.getenv("MONGO_PASSWORD")
DB_URL = os.getenv("MONGO_URL")
DB_NAME = os.getenv("MONGO_DB")

In [5]:
client = pymongo.MongoClient(
    f"mongodb+srv://{DB_USERNAME}:{DB_PASSWORD}@{DB_URL}/?retryWrites=true&w=majority"
)
client

MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True)

In [6]:
db = client[DB_NAME]
db

Database(MongoClient(host=['ac-52gsxzj-shard-00-02.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-00.yi2z9ch.mongodb.net:27017', 'ac-52gsxzj-shard-00-01.yi2z9ch.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-12itgl-shard-0', tls=True), 'mlops-project')

In [7]:
results_dir_path = os.path.join(PROCESSED_DIR_PATH, "results")

In [8]:
groups = [group[:-5] for group in os.listdir(results_dir_path) if group.endswith(".json")]
groups

['2224086878288']

In [9]:
group_id = groups[0]

In [10]:
results = load_results(group_id, results_dir_path)
results

{'1.jpg': [['108.jpg', 0.8954224586486816],
  ['119.jpg', 0.9999999403953552],
  ['127.jpg', 0.9999999403953552],
  ['145.jpg', 0.8954224586486816],
  ['176.jpg', 0.8954224586486816],
  ['192.jpg', 0.861783504486084],
  ['194.jpg', 0.861783504486084],
  ['202.jpg', 0.861783504486084],
  ['24.jpg', 0.861783504486084],
  ['25.jpg', 0.861783504486084],
  ['28.jpg', 0.8954224586486816],
  ['5.jpg', 0.8954224586486816],
  ['64.jpg', 0.9999999403953552],
  ['87.jpg', 0.9999999403953552]],
 '10.jpg': [['154.jpg', 1.000000238418579],
  ['158.jpg', 0.8560693264007568],
  ['178.jpg', 1.000000238418579],
  ['2.jpg', 0.8560693264007568],
  ['35.jpg', 1.000000238418579],
  ['48.jpg', 0.8560693264007568],
  ['56.jpg', 1.000000238418579],
  ['62.jpg', 0.8560693264007568],
  ['83.jpg', 0.8560693264007568]],
 '100.jpg': [['146.jpg', 0.9999997615814209],
  ['78.jpg', 0.9999997615814209],
  ['8.jpg', 0.9999997615814209],
  ['94.jpg', 0.9999997615814209]],
 '101.jpg': [['128.jpg', 0.8809066414833069],
  [

In [13]:
duplicate_images_df = pd.DataFrame(columns=["group_id", "base_image_id", "duplicated_image_id", "probability"])
duplicate_images_df

,group_id,base_image_id,duplicated_image_id,probability


In [14]:
for base_image, duplicated_images in results.items():
    base_image_id = base_image.split(".")[0]
    for duplicated_image in duplicated_images:
        duplicated_image_id = duplicated_image[0].split(".")[0]
        probability = duplicated_image[1]
        
        row = {
            "group_id": group_id,
            "base_image_id": base_image_id,
            "duplicated_image_id": duplicated_image_id,
            "probability": probability
        }
        
        duplicate_images_df = df_duplicates.append(row, ignore_index=True)

/tmp/ipykernel_22/2321687792.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  duplicate_images_df = df_duplicates.append(row, ignore_index=True)
/tmp/ipykernel_22/2321687792.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  duplicate_images_df = df_duplicates.append(row, ignore_index=True)
/tmp/ipykernel_22/2321687792.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  duplicate_images_df = df_duplicates.append(row, ignore_index=True)
/tmp/ipykernel_22/2321687792.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  duplicate_images_df = df_duplicates.append(row, ignore_index=True)
/tmp/ipykernel_22/2321687792.py:14: FutureWarnin

In [16]:
display(duplicate_images_df.head())
display(duplicate_images_df.tail())

,group_id,base_image_id,duplicated_image_id,probability
0,2224086878288,1,108,0.895422
1,2224086878288,1,119,1.000000
2,2224086878288,1,127,1.000000
3,2224086878288,1,145,0.895422
4,2224086878288,1,176,0.895422


,group_id,base_image_id,duplicated_image_id,probability
1786,2224086878288,99,105,1.0
1787,2224086878288,99,110,1.0
1788,2224086878288,99,33,1.0
1789,2224086878288,99,39,1.0
1790,2224086878288,99,39,1.0


In [17]:
duplicates_dict = duplicate_images_df.to_dict("index")
duplicates_dict

{0: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '108',
  'probability': 0.8954224586486816},
 1: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '119',
  'probability': 0.9999999403953552},
 2: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '127',
  'probability': 0.9999999403953552},
 3: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '145',
  'probability': 0.8954224586486816},
 4: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '176',
  'probability': 0.8954224586486816},
 5: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '192',
  'probability': 0.861783504486084},
 6: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '194',
  'probability': 0.861783504486084},
 7: {'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '202',
  'probability':

In [18]:
duplicates_array = [{"date": dt.utcnow(), **obj} for obj in duplicates_dict.values()]
duplicates_array

[{'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634117),
  'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '108',
  'probability': 0.8954224586486816},
 {'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634120),
  'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '119',
  'probability': 0.9999999403953552},
 {'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634120),
  'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '127',
  'probability': 0.9999999403953552},
 {'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634121),
  'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '145',
  'probability': 0.8954224586486816},
 {'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634121),
  'group_id': '2224086878288',
  'base_image_id': '1',
  'duplicated_image_id': '176',
  'probability': 0.8954224586486816},
 {'date': datetime.datetime(2023, 1, 13, 22, 52, 29, 634121),
  '

In [19]:
db["duplicated_images_model_results"].insert_many(duplicates_array)